In [ ]:
import qiskit
import numpy as np
import math

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
qiskit.__version__

In [ ]:
qiskit.__qiskit_version__

In [ ]:
qiskit.IBMQ.save_account('96eb6fdc1c3d8661911c1cf9899e0335dfebbfcaddf93754252ac129d6cfa8a0fb0d5b33439c741cacffab22cda9c28a5adea3ec064cf9ff5919b329f48c89a1')

# QUANTUM ERASER EXPERIMENT

The quantum eraser is an experiment that invites to reflecting on the nature of entanglement. Its most straightforward interpretation would be that, thanks to quantum mechanics, the collapse produced by a measurement could be erased by the application of operators to an entangled element.

Determining the path of a photon after a passing a double slit would supress the interference pattern. Since this knowledge or measurement is achieved through an entangled photon, one could assumed that there is instant communication between detectors. The truth is that in experimental realizations, a good study of the data must be done in order to filter noise and determine the correct timing of events.

## A CLASSICAL EXPERIMENT THAT EMULATES A QUANTUM ERASER

The setup relies in the properties of light. If you have a laser, chances are that the emission will be linearly polarized. To emulate a superposition of states, a quarter-wave plate is used, so the light beam becomes circularly polarized (your state would be 1/Sqrt(2)( |L> + |R> ).

Pointing this beam to a double slit will generate an interference pattern in a screen placed behind. Interference is ruled by phase difference between waves arriving to the same point.

Now, if a polarizer was placed after each slit, and their axis were perpendicular, that would mean we know what path did each wave chose before arriving to the screen. This measurement destroys the interference pattern and a light spot is observed.

Is it possible to erase this 'measurement'? If another polarizer is placed after the previous two, with its axis forming 45° to theirs (remember they were perpendicular to each other), the components surviving this polarizer would make the original slit indistinguishable, thus restoring the interference pattern.

## REPRODUCING THIS EXPERIMENT WITH QUANTUM CIRCUITS

Let us build qiskit circuits that take the ideas from that previous experiment. Some theoretical computation will be included.

Our light beam will be q_0. At some moment there will be some divergence because we have to rely on entanglement instead of having polarizers.

The laser with linear polarization will be |0>. The quarter-wave plate, that changes the state to a different base, will be emulated by a Hadamard gate, which will deliver q_0 = |+> = (1/Sqrt(2))( |0> + |1> ).

In [ ]:
bra_0 = np.array([[1],[0]], dtype=np.complex_)
H = (1/math.sqrt(2))*np.array([[1,1],[1,-1]])
q_0 = H.dot(bra_0)
np.set_printoptions(suppress=True)
print('Theoretical state of q_0:\n', q_0)

In [ ]:
backend = qiskit.Aer.get_backend('statevector_simulator'); # Tell Qiskit how to simulate our circuit

qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state);           # Print the result

The next step would be passing it through the double slit. As stated before, the interference originated from a double slit comes from the phase difference between waves crossing different paths. Here we apply an operator that gets the |+> we have and adds a phase within its components |0> and |1>. The appropriate operator to act in such a base is R_z(phi).

In [ ]:
print('Theory\n','Before applying the rotation: \n', q_0, '\n')

def Rz(phi):
    return np.array([[1, 0],[0, math.e**(1j*phi)]], dtype=np.complex_)

# We will try different phases
q_0_a = Rz(np.pi/2).dot(q_0)
print('After applying a PI/2 rotation \n', q_0_a,'\n')

q_0_b = Rz(np.pi).dot(q_0)
print('After applying a PI rotation \n', q_0_b,'\n')

q_0_c = Rz(np.pi/3).dot(q_0)
print('After applying a PI/3 rotation \n', q_0_c,)

In [ ]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/2, qreg_q[0])

state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi, qreg_q[0])

state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])

state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state);           # Print the result

We are ready now to create the interference. As the documentation suggests, this is represented by applying another Hadamard gate to our circuit. Let us recall that two H operators are just the identity.

In [ ]:
print('Theory\n','Before interfering, the states of q_0 are: \n Pi/2 rotation\n', q_0_a, '\n\n Pi rotation\n', q_0_b, '\n\n Pi/3 rotation\n',
      q_0_c, '\n\nThe interference result will depend'
      ' on phi, which would prove the pattern depending on the phase, that is why we show the three cases.')

q_0_aa = H.dot(q_0_a)
q_0_bb = H.dot(q_0_b)
q_0_cc = H.dot(q_0_c)

print('\nAfter the H interference, the states of q_0 are: \n\n', 'Pi/2 rotation\n', q_0_aa, '\n\n Pi rotation\n', q_0_bb, '\n\n Pi/3 rotation\n',
      q_0_cc)

Let us see if the quantum circuits prove these results:

In [ ]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/2, qreg_q[0])
circuit.h(qreg_q[0])

state_a = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state_a);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi, qreg_q[0])
circuit.h(qreg_q[0])

state_b = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state_b);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])
circuit.h(qreg_q[0])

state_c = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_0: ',state_c);           # Print the result

And show the probabilities associated to measuring either |0> or |1>:

In [ ]:
Prob_pi_half_0 = np.abs(np.array([1,0], dtype=np.complex_).dot(state_a))**2
print('Probability of |0> when applying a Pi/2 rotation: ', Prob_pi_half_0)
Prob_pi_half_1 = np.abs(np.array([0,1], dtype=np.complex_).dot(state_a))**2
print('Probability of |1> when applying a Pi/2 rotation: ', Prob_pi_half_1)

Prob_pi_0 = np.abs(np.array([1,0], dtype=np.complex_).dot(state_b))**2
print('\nProbability of |0> when applying a Pi rotation: ', Prob_pi_0)
Prob_pi_1 = np.abs(np.array([0,1], dtype=np.complex_).dot(state_b))**2
print('Probability of |1> when applying a Pi rotation: ', Prob_pi_1)

Prob_pi_third_0 = np.abs(np.array([1,0], dtype=np.complex_).dot(state_c))**2
print('\nProbability of |0> when applying a Pi/3 rotation: ', Prob_pi_third_0)
Prob_pi_third_1 = np.abs(np.array([0,1], dtype=np.complex_).dot(state_c))**2
print('Probability of |1> when applying a Pi/3 rotation: ', Prob_pi_third_1)

We can see that Prob_|0> = 1/2 (1 + cos(Phi)), the expected theoretical result.

As a reminder, the laser setup can show the phase difference as a distance from the central axis (in the screen), showing an interference. Here we will have to simulate different phases and build a histogram to show the equivalent interference pattern, according to Phi.

### THE WAY TO KNOW WHICH SLIT WAS CHOSEN, THE ENTANGLEMENT

The documentation shows that the way to build an entangled state with a second qubit is through the application of a CNOT gate. To achieve the maximum entanglement, the sequence should be H - CNOT, which defines a Bell state, thus entangling before the Rz(Phi) operator.

However, we can see that, for our classical laser, the knowledge of the path came after the slit, where the polarizators were placed. Can we place our CNOT after the rotation? Let us study the commutator of these operators

In [ ]:
print('We will use Pi/3 to avoid any cancellation\n')

CNOT = np.array([[1, 0, 0, 0],[0, 0, 0, 1],[0, 0, 1, 0],[0, 1, 0, 0]])
print('CNOT =\n', CNOT)
print('\nThe rotation acts on q_0 but not in q_1, the qubit we are goint to entangle to it'
      '\nA kronecker product will be executed against the identity matrix, respecting the q_1 x q_0 notation')

new_Rz = np.kron(np.array([[1, 0],[0, 1]]), Rz(np.pi/3))
print('\nRz_4 = I x Rz =\n',new_Rz)

print('\nCNOT . Rz_4 =\n', CNOT.dot(new_Rz))
print('\nRz_4 . CNOT =\n', new_Rz.dot(CNOT))

print('\nAs we can see, the operators commute, so we will place the CNOT after the rotation, just to stick to'
     'the laser experimental setup.')

We proceed to include a q_1 = |0> qubit in the circuit. From now one, to simplify the book, we will stick to the rotation of Pi/3 in the theoretical study.

We locate ourselves after the rotation and before the second H gate is applied. There are no changes in q_1 yet. The Kronecker product of q_1 x q_0 will be:

In [ ]:
q_1 = np.array([[1],[0]])
print('q_0 = \n', q_0_c)
print('q_1 = \n', q_1)

q_1xq_0 = np.kron(q_1, q_0_c)

print('Composed state q_1 x q_0 = \n', q_1xq_0)

Using qiskit circuit simulator:

In [ ]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])


state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_1 x q_0: \n',state);           # Print the result

We are ready to place the CNOT after the rotation. Theoretically, this should deliver:

In [ ]:
entangled_q1q0 = CNOT.dot(q_1xq_0)
print('Entangled state: \n', entangled_q1q0)

And the qiskit circuit:

In [ ]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])

state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('Entangled state: \n',state);           # Print the result

Now is the moment to see if, after getting knowledge from the entanglement, will have lost any phase influence in the output, destroying the interference pattern.

Theoretically:

In [ ]:
print('The H gate acts only on q_0, we have to do the Kronecker product with the identity again')

new_H = np.kron(np.array([[1, 0],[0, 1]]), H)
print('\nThe new H matrix is:\n', new_H)

print('\n If we now apply it to the entangled state:\n')
interfered_entangled_q1q0 = new_H.dot(entangled_q1q0)
print('The state we have after doing this is:\n', interfered_entangled_q1q0)

To obtain the probabilities of measuring |0> or |1> for q_0 first we have to understand how the base for composed states works:

In [ ]:
zero = np.array([[1],[0]])
one = np.array([[0],[1]])

z_z = np.kron(zero, zero)
z_o = np.kron(zero, one)
o_z = np.kron(one, zero)
o_o = np.kron(one, one)

print('|00> =\n', z_z)
print('\n|01> =\n', z_o)
print('\n|10> =\n', o_z)
print('\n|11> =\n', o_o)

In [ ]:
Prob_entangled_00 = np.abs(z_z.conjugate().transpose().dot(interfered_entangled_q1q0))**2
Prob_entangled_01 = np.abs(z_o.conjugate().transpose().dot(interfered_entangled_q1q0))**2
Prob_entangled_10 = np.abs(o_z.conjugate().transpose().dot(interfered_entangled_q1q0))**2
Prob_entangled_11 = np.abs(o_o.conjugate().transpose().dot(interfered_entangled_q1q0))**2
print('\nProbability of |00> when applying a Pi/3 rotation and then entangle: ', Prob_entangled_00)
print('\nProbability of |01> when applying a Pi/3 rotation and then entangle: ', Prob_entangled_01)
print('\nProbability of |10> when applying a Pi/3 rotation and then entangle: ', Prob_entangled_10)
print('\nProbability of |11> when applying a Pi/3 rotation and then entangle: ', Prob_entangled_11)

Yes, now we can measure 4 states, since we have two qubits. The merge the previous work with this, if we had a second qubit q_1 that had no relationship with q_0, initialized as q_1 = |0>, the previous probability measurents for Rz(Pi/3) would be |00> -> 0.75 and |01> -> 0.25, with the other two equal to 0, since no change is done to q_1

Now, doing the same with qiskit circuit simulator:

In [ ]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])

state = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('Entangled state after the interference: \n',state);           # Print the result

Which is the same that we obtained theoretically. Thus, the same probabilities arise.

### ERASING THE PATH KNOWLEDGE

To sum up what we have until now, we have passed from an interference pattern that defined as Prob_|0> = 1/2 (1 + cos(Phi)) to, after entangling the q_0 with a second qubit, destroying this pattern. We exemplified this with a rotation of Pi/3, where Prob_|00> = 0.75 and Prob_|01> = 0.25 before the entanglement but this moved to Prob_|00> = 0.25 after entangling q_0 with q_1. This behavior is the same for any phase.

We move on now to erasing this behavior. The laser experiment erased the polarizators influences by placing another one with an axis 45° to theirs. Since we assumed that our polarizators equivalent would be the entangled q_1, we must now act on it to produce some change.

Let us recall that q_1 is defined with respect to q_0 as a consequence of the entanglement. But this is a two-way road. Can we transform q_1 as that 45° polarizator did so q_0 measurements will be affected by the Rz(Phi) rotation again? An obvious candidate to scramble with q_1 is the H gate.

Theoretically:

In [ ]:
print('The new Hadamard gate only acts on q_1, the Kronecker product will now be different,\nnothing is directly acting on q_0')

new_H1 = np.kron(H,np.array([[1, 0],[0,1]]))
print('\nOur new H is:\n', new_H1)

print('\nLetting this act on the entangled state we had at the end of the previous circuit:\n')
erased_state = new_H1.dot(interfered_entangled_q1q0)
print('The state vector after the erasure is:\n', erased_state)

The probability of each measurement is now:

In [ ]:
Prob_erased_00 = np.abs(z_z.conjugate().transpose().dot(erased_state))**2
Prob_erased_01 = np.abs(z_o.conjugate().transpose().dot(erased_state))**2
Prob_erased_10 = np.abs(o_z.conjugate().transpose().dot(erased_state))**2
Prob_erased_11 = np.abs(o_o.conjugate().transpose().dot(erased_state))**2
print('\nProbability of |00> after erasing: ', Prob_erased_00)
print('\nProbability of |01> after erasing: ', Prob_erased_01)
print('\nProbability of |10> after erasing: ', Prob_erased_10)
print('\nProbability of |11> after erasing: ', Prob_erased_11)

Though this theoretical computation is only for Rz(Pi/3), the three rotations we previously saw will be done again in qiskit, as it is easy to implement.

In [ ]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.reset(qreg_q[1])
circuit.h(qreg_q[0])
circuit.rz(pi/2, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])

state_a = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_1xq_0: \n',state_a);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.reset(qreg_q[1])
circuit.h(qreg_q[0])
circuit.rz(pi, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])

state_b = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_1xq_0: \n',state_b);           # Print the result

circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.reset(qreg_q[1])
circuit.h(qreg_q[0])
circuit.rz(pi/3, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])

state_c = qiskit.execute(circuit,backend).result().get_statevector(); # Execute the circuit
circuit_1 = qiskit.visualization.circuit_drawer(circuit)
print(circuit_1)
print('State of q_1xq_0: \n',state_c);           # Print the result

Now let us check the probabilities associated to each state for the circuits:

In [ ]:
print('For the first circuit:')
Prob_erased_00 = np.abs(z_z.conjugate().transpose().dot(state_a))**2
Prob_erased_01 = np.abs(z_o.conjugate().transpose().dot(state_a))**2
Prob_erased_10 = np.abs(o_z.conjugate().transpose().dot(state_a))**2
Prob_erased_11 = np.abs(o_o.conjugate().transpose().dot(state_a))**2
print('\nProbability of |00> after erasing: ', Prob_erased_00)
print('\nProbability of |01> after erasing: ', Prob_erased_01)
print('\nProbability of |10> after erasing: ', Prob_erased_10)
print('\nProbability of |11> after erasing: ', Prob_erased_11)

print('----------------------\nFor the second circuit:')
Prob_erased_00 = np.abs(z_z.conjugate().transpose().dot(state_b))**2
Prob_erased_01 = np.abs(z_o.conjugate().transpose().dot(state_b))**2
Prob_erased_10 = np.abs(o_z.conjugate().transpose().dot(state_b))**2
Prob_erased_11 = np.abs(o_o.conjugate().transpose().dot(state_b))**2
print('\nProbability of |00> after erasing: ', Prob_erased_00)
print('\nProbability of |01> after erasing: ', Prob_erased_01)
print('\nProbability of |10> after erasing: ', Prob_erased_10)
print('\nProbability of |11> after erasing: ', Prob_erased_11)

print('----------------------\nFor the third circuit:')
Prob_erased_00 = np.abs(z_z.conjugate().transpose().dot(state_c))**2
Prob_erased_01 = np.abs(z_o.conjugate().transpose().dot(state_c))**2
Prob_erased_10 = np.abs(o_z.conjugate().transpose().dot(state_c))**2
Prob_erased_11 = np.abs(o_o.conjugate().transpose().dot(state_c))**2
print('\nProbability of |00> after erasing: ', Prob_erased_00)
print('\nProbability of |01> after erasing: ', Prob_erased_01)
print('\nProbability of |10> after erasing: ', Prob_erased_10)
print('\nProbability of |11> after erasing: ', Prob_erased_11)

Now, as it can be noted, the pattern game has changed. If, when we had a qubit, Prob_|0> = 1/2 (1 + cos(Phi)), now that we have two of them entangled, we can see that Prob_|00> = Prob_|11> = 1/4 (1 + cos(Phi))

However, as we can clearly see, we went from a total loss of the phase information regarding the rotation to a recovery of the phase afterwards, what some would loosely claim 'taping into the past'.

Another relevant subject would be the placement of the H gate in q_1. Taking into account that the qubits are entangled, is it relevant its location, before or after the interference H gate in q_0? For this, we study the commutation.

In [ ]:
print('H gate in q_0:\n', new_H)
print('\nH gate in q_1:\n', new_H1)

print('\nH_0 . H_1\n', new_H.dot(new_H1))
print('\nH_1 . H_0\n', new_H1.dot(new_H))

The operators commute and there is no difference in when applying each.

# IMPLEMENTATION IN A QUANTUM COMPUTER

Now that we know that the theory is sound and qiskit can simulate circuits that are in agreement with it, it is the moment to get real counts and prove that this setting can actually be realized in a quantum computer.

In [ ]:
from qiskit import IBMQ
import qiskit.tools.jupyter
%matplotlib inline

IBMQ.load_account()

%qiskit_backend_overview

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_london')
backend

Let us do the interference circuit first

In [ ]:
import random as rd
rd.seed(42)
# We will always use the same set of phases
phi = [pi * rand_numb / 100 for rand_numb in rd.sample(range(101), 100)]
# phi = [pi/3, pi/4]

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

# To create the interference pattern, we need a big amount of different phases.
# Since the measurement depends on the cos(phi), we pull random numbers in the interval phi belongs to [0,Pi]
measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=14);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we destroy the interference by entangling the qubits

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we place the H gate as quantum eraser

Here is where things get tricky, and the reason why in the previous circuits there were measurements for both qubits. It is clear that to see the interference pattern of a double slit, we would need to do that measurement, thus why measuring q_1 is necessary.

Why? Because in the previous circuit, for each shot, when we measure we are determining the projection of q_1 in the [|0>, |1>] basis, which is the same as saying we knew the slit.

Now, after placing a H in the q_1, and measuring the system when it has moved to the [|+>,|->] base, that knowledge is lost. However, if we did not measure q_1, it would get the result for both possibilities, rendering a Prob_|0> = 1/4 (1 + cos(Phi)) + 1/4(1-cos(Phi)) = 1/2, thus adding the interference patterns of the possibilities given by q_1 into one indistinguishable spot.

In [ ]:
%%capture

qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.h(qreg_q[1]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

In [ ]:
print('THE END')

Let us do the interference circuit first

In [ ]:
import random as rd
rd.seed(42)
# We will always use the same set of phases
#phi = [pi * rand_numb / 100 for rand_numb in rd.sample(range(101), 100)]
phi = [pi/3, pi/4]

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

# To create the interference pattern, we need a big amount of different phases.
# Since the measurement depends on the cos(phi), we pull random numbers in the interval phi belongs to [0,Pi]
measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we destroy the interference by entangling the qubits

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we place the H gate as quantum eraser

Here is where things get tricky, and the reason why in the previous circuits there were measurements for both qubits. It is clear that to see the interference pattern of a double slit, we would need to do that measurement, thus why measuring q_1 is necessary.

Why? Because in the previous circuit, for each shot, when we measure we are determining the projection of q_1 in the [|0>, |1>] basis, which is the same as saying we knew the slit.

Now, after placing a H in the q_1, and measuring the system when it has moved to the [|+>,|->] base, that knowledge is lost. However, if we did not measure q_1, it would get the result for both possibilities, rendering a Prob_|0> = 1/4 (1 + cos(Phi)) + 1/4(1-cos(Phi)) = 1/2, thus adding the interference patterns of the possibilities given by q_1 into one indistinguishable spot.

In [ ]:
%%capture

qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.h(qreg_q[1]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

In [ ]:
print('THE END')

Let us do the interference circuit first

In [ ]:
import random as rd
rd.seed(42)
# We will always use the same set of phases
#phi = [pi * rand_numb / 100 for rand_numb in rd.sample(range(101), 100)]
phi = [pi/3, pi/4]

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

# To create the interference pattern, we need a big amount of different phases.
# Since the measurement depends on the cos(phi), we pull random numbers in the interval phi belongs to [0,Pi]
measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we destroy the interference by entangling the qubits

In [ ]:
%%capture
qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

Now we place the H gate as quantum eraser

Here is where things get tricky, and the reason why in the previous circuits there were measurements for both qubits. It is clear that to see the interference pattern of a double slit, we would need to do that measurement, thus why measuring q_1 is necessary.

Why? Because in the previous circuit, for each shot, when we measure we are determining the projection of q_1 in the [|0>, |1>] basis, which is the same as saying we knew the slit.

Now, after placing a H in the q_1, and measuring the system when it has moved to the [|+>,|->] base, that knowledge is lost. However, if we did not measure q_1, it would get the result for both possibilities, rendering a Prob_|0> = 1/4 (1 + cos(Phi)) + 1/4(1-cos(Phi)) = 1/2, thus adding the interference patterns of the possibilities given by q_1 into one indistinguishable spot.

In [ ]:
%%capture

qreg_q = QuantumRegister(2, 'q');
creg_c = ClassicalRegister(2, 'c');

measurements = []
for i, phase in enumerate(phi):
    circuit = QuantumCircuit(qreg_q, creg_c);

    circuit.reset(qreg_q[0]);
    circuit.h(qreg_q[0]);
    circuit.rz(phase, qreg_q[0]);
    circuit.cx(qreg_q[0], qreg_q[1]);
    circuit.h(qreg_q[0]);
    circuit.h(qreg_q[1]);
    circuit.measure(qreg_q[0], creg_c[0]);
    circuit.measure(qreg_q[1], creg_c[1]);
    if i==0: # We draw the circuit once
        circuit_1 = qiskit.visualization.circuit_drawer(circuit);
    execution = qiskit.execute(circuit, backend, shots=8);
    data = dict(execution.result().get_counts());
    data.update({'phi': phase});
    measurements.append(data);

In [ ]:
print('This is a circuit with one of the multiple phases we are implementing:\n')
print(circuit_1)

In [ ]:
measurements

### Plotting histogram of measurements


In [ ]:
import matplotlib.pyplot as plt
meas_00 = []
meas_01 = []
meas_10 = []
meas_11 = []
phi = np.array([])
for measurement in measurements:
    meas_00.append(measurement['00'])
    meas_01.append(measurement['01'])
    meas_11.append(measurement['11'])
    meas_10.append(measurement['10'])
    np.extend(phi,measurement['phi'])
    
ax = plt.subplot(111)
ax.bar(phi-0.02, meas_00, width=0.01, color='b', align='center', label='00')
ax.bar(phi+0.02, meas_11, width=0.01, color='r', align='center', label='11')
ax.bar(phi-0.01, meas_01, width=0.01, color='g', align='center', label='01')
ax.bar(phi+0.01, meas_10, width=0.01, color='y', align='center', label='10')


ax.legend()
ax.set_xlabel('phi')
plt.show()

In [ ]:
print('THE END')